In [ ]:
import numpy as np
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

# 导入数据
data = np.loadtxt('[1](https://exoplanets.nasa.gov/exoplanet-catalog/6081/ogle-2005-blg-390l-b/)')
# 提取时间、亮度和误差
t = data[:,0]
m = data[:,1]
sigma = data[:,2]
# 定义参数的边界
bounds = [(0, 4000), # t0
          (0, 1), # u0
          (0, 1000), # tE
          (0, 1), # q
          (0, 10), # s
          (0, 2*np.pi), # alpha
          (0, 0.1)] # rho
# 用差分进化算法拟合参数
result = differential_evolution(objective_function, bounds, args=(t, m, sigma))
# 打印最优参数和目标函数值
print(result.x)
print(result.fun)
# 画出拟合后的光变曲线
plt.errorbar(t, m, yerr=sigma, fmt='.', label='Data')
plt.plot(t, lens_equation(t, *result.x), 'r-', label='Model')
plt.xlabel('Time (days)')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.legend()
plt.show()


In [3]:
# 导入所需的库
import numpy as np
from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt

# 定义透镜方程，计算背景恒星的亮度
def lens_equation(t, t0, u0, tE, q, s, alpha, rho):
    # 计算背景恒星到透镜天体的角距离
    tau = (t - t0) / tE
    u = np.sqrt(u0**2 + tau**2)
    # 计算行星到透镜天体的角距离
    x = s * np.cos(alpha) - tau * np.sin(alpha)
    y = s * np.sin(alpha) + tau * np.cos(alpha)
    v = np.sqrt(x**2 + y**2)
    # 计算背景恒星的放大率
    A = (u**2 + 2) / (u * np.sqrt(u**2 + 4)) + q * (v**2 + 2) / (v * np.sqrt(v**2 + 4))
    # 考虑有限源的效应
    if u < rho or v < rho:
        # 用数值积分的方法计算有限源的放大率
        A = finite_source_magnification(t, t0, u0, tE, q, s, alpha, rho)
    # 返回背景恒星的亮度，即放大率的对数
    return np.log(A)

# 定义有限源的放大率的计算方法，用数值积分的方法
def finite_source_magnification(t, t0, u0, tE, q, s, alpha, rho):
    # 定义积分的上下限和步长
    lower = -rho
    upper = rho
    step = 0.001
    # 定义积分的网格
    x = np.arange(lower, upper, step)
    y = np.arange(lower, upper, step)
    xx, yy = np.meshgrid(x, y)
    # 计算网格上每一点的放大率
    zz = point_source_magnification(t, t0, u0, tE, q, s, alpha, xx, yy)
    # 计算网格上每一点的亮度，即放大率的平方
    zz = zz**2
    # 计算网格上每一点的面积，即步长的平方
    area = step**2
    # 计算有限源的总亮度，即网格上所有点的亮度乘以面积的和
    total_brightness = np.sum(zz) * area
    # 计算有限源的平均亮度，即总亮度除以源的面积
    mean_brightness = total_brightness / (np.pi * rho**2)
    # 计算有限源的放大率，即平均亮度的平方根
    A = np.sqrt(mean_brightness)
    # 返回有限源的放大率
    return A

# 定义点源的放大率的计算方法，用透镜方程的公式
def point_source_magnification(t, t0, u0, tE, q, s, alpha, x, y):
    # 计算点源到透镜天体的角距离
    tau = (t - t0) / tE
    u = np.sqrt(u0**2 + tau**2)
    # 计算点源到行星的角距离
    xp = s * np.cos(alpha) - tau * np.sin(alpha)
    yp = s * np.sin(alpha) + tau * np.cos(alpha)
    v = np.sqrt(xp**2 + yp**2)
    # 计算点源的放大率
    A = (u**2 + 2) / (u * np.sqrt(u**2 + 4)) + q * (v**2 + 2) / (v * np.sqrt(v**2 + 4))
    # 返回点源的放大率
    return A

# 定义目标函数，即模型与数据之间的差异
def objective_function(params, t, m, sigma):
    # 解包参数
    t0, u0, tE, q, s, alpha, rho = params
    # 计算模型的亮度
    m_model = lens_equation(t, t0, u0, tE, q, s, alpha, rho)
    # 计算最小二乘差
    chi2 = np.sum(((m - m_model) / sigma)**2)
    # 返回目标函数值
    return chi2

# 导入数据
#data = np.loadtxt('[1](https://exoplanets.nasa.gov/exoplanet-catalog/6081/ogle-2005-blg-390l-b/)')
# 定义时间范围从3480到3612，每隔0.5天取一个值
t = np.arange(3480, 3612.5, 0.5)

# 对应的m为16
m = np.full_like(t, 16)

# 对应的sigma为0.02
sigma = np.full_like(t, 0.02)

# 提取时间、亮度和误差
#t = data[:,0]
#m = data[:,1]
#sigma = data[:,2]
# 定义参数的边界
bounds = [(0, 4000), # t0
          (0, 1), # u0
          (0, 1000), # tE
          (0, 1), # q
          (0, 10), # s
          (0, 2*np.pi), # alpha
          (0, 0.1)] # rho
# 用差分进化算法拟合参数
result = differential_evolution(objective_function, bounds, args=(t, m, sigma))
# 打印最优参数和目标函数值
print(result.x)
print(result.fun)
# 画出拟合后的光变曲线
plt.errorbar(t, m, yerr=sigma, fmt='.', label='Data')
plt.plot(t, lens_equation(t, *result.x), 'r-', label='Model')
plt.xlabel('Time (days)')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.legend()
plt.show()


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'

In [5]:
# 定义时间范围从3480到3612，每隔0.5天取一个值
t = np.arange(3480, 3612.5, 0.5)

# 对应的m为16
m = np.full_like(t, 16)

# 对应的sigma为0.02
sigma = np.full_like(t, 0.02)

t,m,sigma

(array([3480. , 3480.5, 3481. , 3481.5, 3482. , 3482.5, 3483. , 3483.5,
        3484. , 3484.5, 3485. , 3485.5, 3486. , 3486.5, 3487. , 3487.5,
        3488. , 3488.5, 3489. , 3489.5, 3490. , 3490.5, 3491. , 3491.5,
        3492. , 3492.5, 3493. , 3493.5, 3494. , 3494.5, 3495. , 3495.5,
        3496. , 3496.5, 3497. , 3497.5, 3498. , 3498.5, 3499. , 3499.5,
        3500. , 3500.5, 3501. , 3501.5, 3502. , 3502.5, 3503. , 3503.5,
        3504. , 3504.5, 3505. , 3505.5, 3506. , 3506.5, 3507. , 3507.5,
        3508. , 3508.5, 3509. , 3509.5, 3510. , 3510.5, 3511. , 3511.5,
        3512. , 3512.5, 3513. , 3513.5, 3514. , 3514.5, 3515. , 3515.5,
        3516. , 3516.5, 3517. , 3517.5, 3518. , 3518.5, 3519. , 3519.5,
        3520. , 3520.5, 3521. , 3521.5, 3522. , 3522.5, 3523. , 3523.5,
        3524. , 3524.5, 3525. , 3525.5, 3526. , 3526.5, 3527. , 3527.5,
        3528. , 3528.5, 3529. , 3529.5, 3530. , 3530.5, 3531. , 3531.5,
        3532. , 3532.5, 3533. , 3533.5, 3534. , 3534.5, 3535. , 